In [ ]:
%matplotlib inline

import gym
import matplotlib
import numpy as np
import sys
import time

from collections import defaultdict

from blackjack import BlackjackEnv
import plotting

matplotlib.style.use('ggplot')

In [ ]:
env = BlackjackEnv()

In [ ]:
def make_epsilon_greedy_policy(Q, epsilon, nA):
    """
    Creates an epsilon-greedy policy based on a given Q-function and epsilon.

    Args:
        Q: A dictionary that maps from state -> action-values.
            Each value is a numpy array of length nA (see below)
        epsilon: The probability to select a random action . float between 0 and 1.
        nA: Number of actions in the environment.

    Returns:
        A function that takes the observation as an argument and returns
        the probabilities for each action in the form of a numpy array of length nA.

    """
    def policy_fn(observation):
        A = np.full(nA, epsilon / nA)
        A[Q[observation].argmax()] += 1 - epsilon

        return A

    return policy_fn

In [ ]:
from tqdm.notebook import trange
from collections import namedtuple

EpisodeEntry = namedtuple("EpisodeEntry", "state action reward")


def mc_control_epsilon_greedy(env, num_episodes, discount_factor=1.0, epsilon=0.1):
    """
    Monte Carlo Control using Epsilon-Greedy policies.
    Finds an optimal epsilon-greedy policy.
    
    Args:
        env: OpenAI gym environment.
        num_episodes: Number of episodes to sample.
        discount_factor: Gamma discount factor.
        epsilon: Chance the sample a random action. Float betwen 0 and 1.
    
    Returns:
        A tuple (Q, policy).
        Q is a dictionary mapping state -> action values.
        policy is a function that takes an observation as an argument and returns
        action probabilities
    """

    # Keeps track of sum and count of returns for each state
    # to calculate an average. We could use an array to save all
    # returns (like in the book) but that's memory inefficient.
    returns_sum = defaultdict(float)
    returns_count = defaultdict(float)

    # The final action-value function.
    # A nested dictionary that maps state -> (action -> action-value).
    Q = defaultdict(lambda: np.zeros(env.action_space.n))

    # The policy we're following
    policy = make_epsilon_greedy_policy(Q, epsilon, env.action_space.n)

    for _ in trange(num_episodes):
        # Generate an episode.
        # An episode is an array of (state, action, reward) tuples
        # Same as for MC Prediction.ipynb but using Ɛ-greedy this time
        episode = []
        state = env.reset()
        for t in range(100):
            action_probas = policy(state)
            action = np.random.choice(range(len(action_probas)), p=action_probas)
            next_state, reward, done, _ = env.step(action)
            episode.append(EpisodeEntry(state, action, reward))
            if done:
                break
            state = next_state

        # Again, similar to MC Prediction.ipynb
        # Now instead iterating over all (state, action) pairs
        for state_action_pair in {(x.state, x.action) for x in episode}:
            # Sum up all rewards since the first occurance
            first_occurence_idx = next(
                i
                for i, x in enumerate(episode)
                if (x.state, x.action) == state_action_pair
            )
            G = sum(
                [
                    x.reward * (discount_factor ** i)
                    for i, x in enumerate(episode[first_occurence_idx:])
                ]
            )

            # Calculate average return for this state over all sampled episodes
            returns_sum[state_action_pair] += G
            returns_count[state_action_pair] += 1.0

            state, action = state_action_pair
            Q[state][action] = (
                returns_sum[state_action_pair] / returns_count[state_action_pair]
            )

    return Q, policy

In [ ]:
Q, policy = mc_control_epsilon_greedy(env, num_episodes=500000, epsilon=0.1)

In [ ]:
# For plotting: Create value function from action-value function
# by picking the best action at each state
V = defaultdict(float)
for state, actions in Q.items():
    action_value = np.max(actions)
    V[state] = action_value
plotting.plot_value_function(V, title="Optimal Value Function")